<a href="https://colab.research.google.com/github/jackliusr/data-science/blob/main/split_then_refine_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab contains the Official demo of our AAAI21 paper:

[**Split then Refine: Stacked Attention-guided ResUNets for Blind Single Image Visible Watermark Removal**](https://arxiv.org/abs/2012.07007)

Xiaodong Cun, Chi-Man Pun*

University of Macau

In [3]:
! wget -O car.jpg https://i.i-sgcm.com/new_cars/cars/12258/12258_g34_b.jpg
! pwd 
! ls

--2021-04-16 12:07:36--  https://i.i-sgcm.com/new_cars/cars/12258/12258_g34_b.jpg
Resolving i.i-sgcm.com (i.i-sgcm.com)... 152.199.1.124
Connecting to i.i-sgcm.com (i.i-sgcm.com)|152.199.1.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172986 (169K) [image/jpeg]
Saving to: ‘car.jpg’

car.jpg             100%[===================>] 168.93K  --.-KB/s    in 0.03s   

2021-04-16 12:07:36 (5.29 MB/s) - ‘car.jpg’ saved [172986/172986]

/content
27kpng_model_best.pth.tar     __MACOSX	   val_images
car.jpg			      natural	   val.zip
deep-blind-watermark-removal  natural.zip


In [2]:
# download the necessary componments
! rm -rf *
! git clone https://github.com/vinthony/deep-blind-watermark-removal.git # get code from github
! gdown https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc # get pretrained model
! gdown https://drive.google.com/uc?id=18HaWfYYZCD34VttSjd2at8b9BKdhgVgU && unzip -q val.zip # get validation dataset (2.31G) of 27kpng
! gdown https://drive.google.com/uc?id=1it5oQDRqRzBVieX6jKNmOxj1992f63yM && unzip -q natural.zip # get natural images (0.4G) of 27kpng

# rename natural images
from os import listdir
from os.path import isfile, join
import shutil
filenames = [ shutil.copy(join('./natural', f), join('./natural', f).split('-')[0]+'.jpg') for f in listdir('./natural') if isfile(join('./natural', f)) ]

Cloning into 'deep-blind-watermark-removal'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 84 (delta 23), reused 50 (delta 10), pack-reused 0
Unpacking objects: 100% (84/84), done.
Downloading...
From: https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc
To: /content/27kpng_model_best.pth.tar
131MB [00:01, 99.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=18HaWfYYZCD34VttSjd2at8b9BKdhgVgU
To: /content/val.zip
2.31GB [00:29, 78.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1it5oQDRqRzBVieX6jKNmOxj1992f63yM
To: /content/natural.zip
443MB [00:09, 45.6MB/s]


In [10]:
import os, sys, torch,random,torchvision
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm

sys.path.append('deep-blind-watermark-removal')
sys.path.insert(0,'deep-blind-watermark-removal')

from scripts.utils.imutils import im_to_numpy
import scripts.models as models
import scripts.datasets as datasets
%matplotlib inline
from PIL import Image, ImageChops

def get_jet():
    colormap_int = np.zeros((256, 3), np.uint8)
 
    for i in range(0, 256, 1):
        colormap_int[i, 0] = np.int_(np.round(cm.jet(i)[0] * 255.0))
        colormap_int[i, 1] = np.int_(np.round(cm.jet(i)[1] * 255.0))
        colormap_int[i, 2] = np.int_(np.round(cm.jet(i)[2] * 255.0))

    return colormap_int

def clamp(num, min_value, max_value):
    return max(min(num, max_value), min_value)

def gray2color(gray_array, color_map):
    
    rows, cols = gray_array.shape
    color_array = np.zeros((rows, cols, 3), np.uint8)
 
    for i in range(0, rows):
        for j in range(0, cols):
#             log(256,2) = 8 , log(1,2) = 0 * 8
            color_array[i, j] = color_map[clamp(int(abs(gray_array[i, j])*10),0,255)]
    
    return color_array

class objectview(object):
    def __init__(self, *args, **kwargs):
        d = dict(*args, **kwargs)
        self.__dict__ = d

jet_map = get_jet()

resume_path = '27kpng_model_best.pth.tar' # path of pretrained model
samples = [320,1364,1868] #random.sample(range(4000), 1) # show random sample 

data_config  = objectview({'input_size':256,
                            'limited_dataset':0,
                            'normalized_input':False,
                            'data_augumentation':False,
                            'base_dir':'.',
                            'data':'_images'})

val_loader = torch.utils.data.DataLoader(datasets.COCO('val',config=data_config,sample=samples))

print('input          | target              | coarser            | final')
print('----------------------------------------------------------------------------')
print('predicted mask | predicted watermark | coarser difference | final difference')

with torch.no_grad():

      model = models.__dict__['vvv4n']().cuda()
      model.load_state_dict(torch.load(resume_path)['state_dict'])
      model.eval()
     
      im  = Image.open("car.jpg").convert("RGB")
      im = torchvision.transforms.ToTensor()(im).cuda()
      imoutput,immask,imwatermark = model(im)
      imcoarser,imrefine,imwatermark = imoutput[1]*immask + im*(1-immask),imoutput[0]*immask + im*(1-immask),imwatermark*immask
      imfinal = np.concatenate([imcoarser,imrefine,imwatermark],axis=0)
      plt.imshow(imfinal,vmin=0.0,vmax=255.0)


total Dataset of val_images is :  3
input          | target              | coarser            | final
----------------------------------------------------------------------------
predicted mask | predicted watermark | coarser difference | final difference


RuntimeError: ignored